In [46]:
import pandas as pd
import numpy as np

In [84]:
df = pd.read_csv('lyrics/csv/Beyonce.csv')
print(df['Lyric'][0])

beyoncé i've been drinkin' i've been drinkin' i get filthy when that liquor get into me i've been thinkin' i've been thinkin' why can't i keep my fingers off it baby i want you nownow why can't i keep my fingers off you baby i want you na na cigars on ice cigars on ice feelin' like an animal with these cameras all in my grill flashin' lights flashin' lights you got me faded faded faded baby i want you na na can't keep your eyes off my fatty daddy i want you na na drunk in love i want you  pre beyoncé we woke up in the kitchen saying how the hell did this shit happen oh baby drunk in love we be all night last thing i remember is our beautiful bodies grindin' up in that club drunk in love   beyoncé we be all night love love we be all night love love   beyoncé we be all night and everything all right no complaints from my body so fluorescent under these lights boy i'm drinkin' park it in my lot 7 i'm rubbin' on it rubrubbin' if you scared call that reverend boy i'm drinkin' get my brain r

In [85]:
lyrics = " ".join(df['Lyric'])

chars = list(set(lyrics))
vocab_size = len(chars)
print(''.join(sorted(chars)))


 '0456789abcdefghijklmnopqrstuvwxyz¾àáâãçèéêíïñóúɔɛɲеẹọ   


In [86]:
#create a mapping

In [87]:
stoi = {ch: i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
# idea - use sentencepiece - sub-word tokenizer
encode = lambda s: [stoi[c] for c in s] # encoder
decode = lambda l: ''.join([itos[i] for i in l]) # decoder
print(encode('hello boys'))
print(decode(encode('hello boys')))

[42, 51, 36, 36, 4, 2, 52, 4, 33, 48]
hello boys


In [88]:
# now lets create one string of text for the whole lyrics
lyrics = " ".join(df['Lyric'])
print(lyrics)

beyoncé i've been drinkin' i've been drinkin' i get filthy when that liquor get into me i've been thinkin' i've been thinkin' why can't i keep my fingers off it baby i want you nownow why can't i keep my fingers off you baby i want you na na cigars on ice cigars on ice feelin' like an animal with these cameras all in my grill flashin' lights flashin' lights you got me faded faded faded baby i want you na na can't keep your eyes off my fatty daddy i want you na na drunk in love i want you  pre beyoncé we woke up in the kitchen saying how the hell did this shit happen oh baby drunk in love we be all night last thing i remember is our beautiful bodies grindin' up in that club drunk in love   beyoncé we be all night love love we be all night love love   beyoncé we be all night and everything all right no complaints from my body so fluorescent under these lights boy i'm drinkin' park it in my lot 7 i'm rubbin' on it rubrubbin' if you scared call that reverend boy i'm drinkin' get my brain r

In [89]:
import torch
data = torch.tensor(encode(lyrics), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([804702]) torch.int64
tensor([52, 51, 33,  4, 46,  8, 12,  2, 23, 39,  9, 51,  2, 52, 51, 51, 46,  2,
        13, 49, 23, 46, 41, 23, 46, 39,  2, 23, 39,  9, 51,  2, 52, 51, 51, 46,
         2, 13, 49, 23, 46, 41, 23, 46, 39,  2, 23,  2, 43, 51, 29,  2, 55, 23,
        36, 29, 42, 33,  2,  5, 42, 51, 46,  2, 29, 42, 10, 29,  2, 36, 23, 17,
        11,  4, 49,  2, 43, 51, 29,  2, 23, 46, 29,  4,  2, 20, 51,  2, 23, 39,
         9, 51,  2, 52, 51, 51, 46,  2, 29, 42, 23, 46, 41, 23, 46, 39,  2, 23,
        39,  9, 51,  2, 52, 51, 51, 46,  2, 29, 42, 23, 46, 41, 23, 46, 39,  2,
         5, 42, 33,  2,  8, 10, 46, 39, 29,  2, 23,  2, 41, 51, 51, 50,  2, 20,
        33,  2, 55, 23, 46, 43, 51, 49, 48,  2,  4, 55, 55,  2, 23, 29,  2, 52,
        10, 52, 33,  2, 23,  2,  5, 10, 46, 29,  2, 33,  4, 11,  2, 46,  4,  5,
        46,  4,  5,  2,  5, 42, 33,  2,  8, 10, 46, 39, 29,  2, 23,  2, 41, 51,
        51, 50,  2, 20, 33,  2, 55, 23, 46, 43, 51, 49, 48,  2,  4, 55, 55,  2,
       

In [90]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [91]:
blocksize = 8
train_data[:blocksize+1]

tensor([52, 51, 33,  4, 46,  8, 12,  2, 23])

In [92]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

inputs
torch.Size([4, 8])
tensor([[ 2, 42, 51, 10, 49, 29,  2, 23],
        [46, 51, 51, 13,  2, 10,  2, 52],
        [42,  2, 51,  9, 51, 49, 33,  2],
        [ 4, 29, 29, 10, 39,  2, 41, 51]])
torch.Size([4, 8])
tensor([[42, 51, 10, 49, 29,  2, 23, 48],
        [51, 51, 13,  2, 10,  2, 52, 10],
        [ 2, 51,  9, 51, 49, 33,  2, 20],
        [29, 29, 10, 39,  2, 41, 51, 51]])


In [95]:
# bigram model, for prediction of the next character we look only at the previous character
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

m = BigramLM(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 58])
4àfá 9èídọp9kvẹiktbãiu8'еu8dọɔọúáɔaiâyáxmks rhɛ¾okyorjéa óp0óf eáóélãb8еãoɲâuèeo785ɲuçɛnkâs0¾h5hàkh á


In [96]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)


In [99]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')


    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    print(loss.item())


3.479088544845581
3.4539084434509277
3.6210877895355225
3.6281888484954834
3.497533082962036
3.4403481483459473
3.559251308441162
3.442009210586548
3.551090717315674
3.5468013286590576
3.470301866531372
3.470923900604248
3.5765955448150635
3.4567999839782715
3.514143228530884
3.5529019832611084
3.436112642288208
3.5093777179718018
3.5685555934906006
3.5425727367401123
3.4033544063568115
3.5437870025634766
3.44224214553833
3.491472005844116
3.419121742248535
3.5780327320098877
3.5176284313201904
3.4271554946899414
3.489638090133667
3.4533727169036865
3.4600415229797363
3.476754665374756
3.442924737930298
3.4857492446899414
3.3951525688171387
3.468430757522583
3.399500608444214
3.45414400100708
3.471574544906616
3.4724020957946777
3.526076078414917
3.4531056880950928
3.5094339847564697
3.4498589038848877
3.534937858581543
3.4438018798828125
3.617159843444824
3.5018351078033447
3.511239528656006
3.4400744438171387
3.5722672939300537
3.4739365577697754
3.4706320762634277
3.468597173690796


In [101]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))

4pgimer y gheale prto lste yohi derdiatarecu ohi lourn  ar ke wn hi swhetos shando y woups y o ouber ba wa wak ngs wa itather haulll w ghinoahete gour yong lou wescke fofocé sis l jàllindor ck yox walkenám cot myowelld l apush  la th plllllabyou menout i o an d imarlantous y be gi' cé fareto ca it tp
